In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
#### Import libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pathlib import Path

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression, f_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

path = Path('')

import warnings
warnings.filterwarnings("ignore")
import warnings
warnings.filterwarnings("ignore")


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, cross_val_score


import warnings
warnings.filterwarnings("ignore")

# # Download the property data
# url = 'https://opendata-downloads.s3.amazonaws.com/opa_properties_public.csv'

In [2]:
data = pd.read_csv('opa_properties_public.csv', header=0)
print(data.shape)
data.head()

(581414, 78)


,objectid,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,...,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,lat,lng
0,78579872,2021-07-05 09:29:19,A,NWC OF 24TH & OXFORD ST,53912171.0,SR,VACANT LAND RESIDE < ACRE,1,Single Family,139.0,...,NaN,NaN,NaN,2019.0,NaN,19121.0,RSA5,1001680150,-75.174210,39.979126
1,78579873,2021-05-25 16:02:19,0,NEC LOCUST,53912526.0,590,RES CONDO 5+ STY MASONRY,1,Single Family,8.0,...,1417,NaN,C,1900.0,Y,19103.0,RMX3,1001629392,-75.170423,39.949254
2,78579874,2022-02-17 13:44:33,NaN,"88'2 7/8"" E SEPVIVA",53911650.0,WA0,HSE WORSHIP ALL 1 STY MAS,1,Single Family,160.0,...,NaN,NaN,I,2020.0,NaN,19125.0,RSA5,1001678063,-75.128893,39.978672
3,78579875,2021-10-12 15:40:41,A,"476' 1 1/4"" S WIGARD",53911139.0,JJ1,AMUSE TV/RADIO STA MAS+O,1,Single Family,216.0,...,15,NaN,I,2021.0,NaN,19128.0,RSD3,1001677510,-75.235043,40.045622
4,78579876,NaN,NaN,SWC REED ST,53764990.0,SR,VACANT LAND RESIDE < ACRE,14,NaN,32.0,...,NaN,NaN,I,NaN,NaN,19146.0,RSA5,1001681386,-75.185151,39.934839


1. data in 1949, 2020, 2021, 2022 four years
2. basement 0,1,2,4, A--J transform 1:A, 2:B, 4:D  A--D,1,2,4: Full, E--H: Partial; IJ: unknown_size; 0: zero_size
3. book_and_page column: relationship of order and house price 
4. "central_air" --287316 non-null   Y: 1; N: 0 transform 
5. date_exterior_condition??
6. add fireplace from year 2000
7. fuel column I, blank : Others
8. garage_type 1,2,3 --> A,B,C
9. add house number --A five-digit field which indicates a specific property address on a block. Odd numbers are normally on the North or East side of a street. Even numbers are on the South or West side of a street.
10. location/ mailing zipcode add one column 
11. number_of_rooms: total number of rooms in the structure  89???
12. "quality_grade" categories 
13. data compare: recoding data, sale data, assessment data????
14. sale price units???
15. sale price & market price ????

In [4]:
data.columns

Index(['objectid', 'assessment_date', 'basements', 'beginning_point',
       'book_and_page', 'building_code', 'building_code_description',
       'category_code', 'category_code_description', 'census_tract',
       'central_air', 'cross_reference', 'date_exterior_condition', 'depth',
       'exempt_building', 'exempt_land', 'exterior_condition', 'fireplaces',
       'frontage', 'fuel', 'garage_spaces', 'garage_type',
       'general_construction', 'geographic_ward', 'homestead_exemption',
       'house_extension', 'house_number', 'interior_condition', 'location',
       'mailing_address_1', 'mailing_address_2', 'mailing_care_of',
       'mailing_city_state', 'mailing_street', 'mailing_zip', 'market_value',
       'market_value_date', 'number_of_bathrooms', 'number_of_bedrooms',
       'number_of_rooms', 'number_stories', 'off_street_open',
       'other_building', 'owner_1', 'owner_2', 'parcel_number', 'parcel_shape',
       'quality_grade', 'recording_date', 'registry_number', 'sale_

### 1. select related columns 

In [5]:
# place columns of interest into a list
cols_interest = ['assessment_date', 'basements', 'category_code_description','census_tract',
                 'central_air', 'depth', 'exempt_building', 'exempt_land',
                 'exterior_condition', 'fireplaces', 'frontage', 'fuel', 'garage_spaces',
                 'garage_type', 'general_construction', 'geographic_ward', 'house_extension', 'house_number',
                 'interior_condition', 'market_value', 'market_value_date', 'number_of_bathrooms', 'number_of_bedrooms',
                 'number_of_rooms', 'number_stories', 'off_street_open', 'other_building', 'parcel_number',
                 'parcel_shape','quality_grade', 'recording_date', 'sale_date', 'sale_price', 'separate_utilities',
                 'sewer', 'site_type', 'street_designation', 'street_direction',
                 'taxable_building', 'taxable_land', 'topography', 'total_area', 'total_livable_area',
                 'type_heater', 'unfinished', 'unit', 'utility', 'view_type',
                 'year_built', 'year_built_estimate', 'zip_code', 'lat', 'lng']


len(cols_interest)

53

## Questions????
- delete 'building_code_description'
- delete 'date_exterior_condition' can EDA with 'exterior_condition'
- 'off_street_open' cannot explain (Generally, off street parking means parking vehicles anywhere on the streets like garages, private lots and the drive ways. It can be on indoor and outdoor sites. A typical off street parking facility is as shown below.)
- parcel_number: for tax 
- EDA: 1) Property characteristics; 2)Tax information; 3) ownership information; 4) assessment related 
- geographic_ward?????meaning 
- delete zoning and pin 
- zip code and 'street_code' leave one ??????  --leave zipcode because catogory is relatively small

1.  'zoning'  'zoning', 'pin' unique identify like accoutid 
2.  street_code	street_designation	street_direction	street_name(categorical meaning )

- street_code is numerical or categorical???
- combine zipcode for comparision 



In [6]:
# read in data using columns of interest 
data_step1 = data[cols_interest]
# compute percentage of missing values for each column
data_step1.isnull().sum()/len(data_step1)

assessment_date              0.941171
basements                    0.438185
category_code_description    0.000193
census_tract                 0.000189
central_air                  0.506192
depth                        0.001089
exempt_building              0.000423
exempt_land                  0.000423
exterior_condition           0.047782
fireplaces                   0.007786
frontage                     0.001097
fuel                         0.974443
garage_spaces                0.008084
garage_type                  0.137986
general_construction         0.109917
geographic_ward              0.000189
house_extension              0.954052
house_number                 0.000000
interior_condition           0.049151
market_value                 0.000423
market_value_date            1.000000
number_of_bathrooms          0.007585
number_of_bedrooms           0.007059
number_of_rooms              0.057656
number_stories               0.007053
off_street_open              0.001760
other_buildi

#### Step 2: We dropped columns with more than 90% missing values, as they would not provide robust and useful information for modeling and prediction process.

In [7]:
# drop columns with over 90% missing values
data_step2 = data_step1.drop(['assessment_date', 'fuel', 'house_extension', 'market_value_date', 'other_building', 
                              'quality_grade', 'separate_utilities', 'sewer', 'unfinished', 'unit', 'utility'], axis = 1)

print(len(data_step2.columns))
data_step2.columns

42


Index(['basements', 'category_code_description', 'census_tract', 'central_air',
       'depth', 'exempt_building', 'exempt_land', 'exterior_condition',
       'fireplaces', 'frontage', 'garage_spaces', 'garage_type',
       'general_construction', 'geographic_ward', 'house_number',
       'interior_condition', 'market_value', 'number_of_bathrooms',
       'number_of_bedrooms', 'number_of_rooms', 'number_stories',
       'off_street_open', 'parcel_number', 'parcel_shape', 'recording_date',
       'sale_date', 'sale_price', 'site_type', 'street_designation',
       'street_direction', 'taxable_building', 'taxable_land', 'topography',
       'total_area', 'total_livable_area', 'type_heater', 'view_type',
       'year_built', 'year_built_estimate', 'zip_code', 'lat', 'lng'],
      dtype='object')

In [8]:
data_step2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581414 entries, 0 to 581413
Data columns (total 42 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   basements                  326647 non-null  object 
 1   category_code_description  581302 non-null  object 
 2   census_tract               581304 non-null  float64
 3   central_air                287107 non-null  object 
 4   depth                      580781 non-null  float64
 5   exempt_building            581168 non-null  float64
 6   exempt_land                581168 non-null  float64
 7   exterior_condition         553633 non-null  float64
 8   fireplaces                 576887 non-null  float64
 9   frontage                   580776 non-null  float64
 10  garage_spaces              576714 non-null  float64
 11  garage_type                501187 non-null  object 
 12  general_construction       517507 non-null  object 
 13  geographic_ward            58

In [9]:
data_transform = data_step2.copy()


data_transform['basements'] = data_transform['basements'].replace({'0': 'None', 'A': 'full', 'B': 'full','C': 'full','D': 'full',
                                                         'E': 'partial','F': 'partial','G': 'partial','H': 'partial',
                                                         'J': 'unknown','I': 'unknown'})

data_transform['central_air'] = data_transform['central_air'].replace({'0': 'N', '1': 'Y','Y': 'Y', 'N': 'N'})

data_transform['exterior_condition'] = data_transform['exterior_condition'].astype(object)
data_transform['garage_type'] = data_transform['garage_type'].replace({'0':'None', '1': 'A', '2': 'B','3': 'C'})
data_transform['general_construction'] = data_transform['garage_type'].replace({'0':'None', '1': 'A', '3': 'C','5': 'E',
                                                                                '6':'F', '8':'H', '9':'I' })
data_transform['house_number'] = np.where(data_transform['house_number']%2==0, 'North or East', 'South or West')
data_transform['interior_condition'] = data_transform['interior_condition'].astype(object)


# transform record date/ sale date to year/month 
# calculate difference between 'recording_date' and 'sale_date', add columns names "Sale_Record_dayDiff"
# data_transform['year'] = data_transform.DatetimeIndex(data_transform['ArrivalDate']).year/month
StartDate_Sale = data_transform['sale_date']
EndDate_Record = data_transform['recording_date']
dateDiff = pd.to_datetime(EndDate_Record, errors='coerce') - pd.to_datetime(StartDate_Sale, errors='coerce')
data_transform['Sale_Record_dayDiff'] = dateDiff.dt.days     # transform to float64
# drop two date column 
data_transform = data_transform.drop(['sale_date','recording_date'],axis=1)


data_transform['topography'] = data_transform['topography'].replace({'0':'None'})
data_transform['type_heater'] = data_transform['type_heater'].replace({'0':'None'})
data_transform['view_type'] = data_transform['view_type'].replace({'0':'Not_Applicable'})

# calculate building years
# one column is outlier 
data_transform['year_built'] = data_transform['year_built'].replace({'196Y':'1965'})
data_transform['Built_years'] = 2022-data_transform['year_built'].astype(float)
data_transform = data_transform.drop(['year_built'],axis=1)
print(len(data_transform.columns))
data_transform


41


,basements,category_code_description,census_tract,central_air,depth,exempt_building,exempt_land,exterior_condition,fireplaces,frontage,...,total_area,total_livable_area,type_heater,view_type,year_built_estimate,zip_code,lat,lng,Sale_Record_dayDiff,Built_years
0,full,Single Family,139.0,Y,94.00,188045.0,0.0,1.0,NaN,16.00,...,1542.00,1242.0,A,NaN,NaN,19121.0,-75.174210,39.979126,124.0,3.0
1,None,Single Family,8.0,Y,0.00,0.0,0.0,3.0,0.0,0.00,...,0.00,524.0,NaN,C,Y,19103.0,-75.170423,39.949254,145.0,122.0
2,NaN,Single Family,160.0,Y,50.00,488400.0,0.0,1.0,NaN,NaN,...,800.00,2499.0,A,I,NaN,19125.0,-75.128893,39.978672,154.0,2.0
3,full,Single Family,216.0,Y,109.00,489118.0,81282.0,1.0,0.0,80.00,...,8219.00,2716.0,A,I,NaN,19128.0,-75.235043,40.045622,118.0,1.0
4,NaN,NaN,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,I,NaN,19146.0,-75.185151,39.934839,357.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581409,full,Single Family,320.0,N,90.00,0.0,0.0,4.0,0.0,16.12,...,1450.80,1120.0,H,I,Y,19135.0,-75.056051,40.024381,0.0,72.0
581410,unknown,Single Family,102.0,NaN,90.00,0.0,0.0,4.0,0.0,16.00,...,1440.00,1504.0,NaN,I,Y,19131.0,-75.230492,39.968486,0.0,97.0
581411,NaN,Commercial,802.0,NaN,900.00,26669.0,6640631.0,4.0,0.0,540.00,...,425776.00,1185.0,NaN,I,NaN,19136.0,-75.020044,40.044684,0.0,52.0
581412,NaN,Vacant Land,335.0,NaN,164.29,0.0,96800.0,NaN,0.0,114.47,...,18806.27,0.0,NaN,I,NaN,19111.0,-75.065764,40.052661,0.0,2022.0


In [10]:
data_transform.columns

Index(['basements', 'category_code_description', 'census_tract', 'central_air',
       'depth', 'exempt_building', 'exempt_land', 'exterior_condition',
       'fireplaces', 'frontage', 'garage_spaces', 'garage_type',
       'general_construction', 'geographic_ward', 'house_number',
       'interior_condition', 'market_value', 'number_of_bathrooms',
       'number_of_bedrooms', 'number_of_rooms', 'number_stories',
       'off_street_open', 'parcel_number', 'parcel_shape', 'sale_price',
       'site_type', 'street_designation', 'street_direction',
       'taxable_building', 'taxable_land', 'topography', 'total_area',
       'total_livable_area', 'type_heater', 'view_type', 'year_built_estimate',
       'zip_code', 'lat', 'lng', 'Sale_Record_dayDiff', 'Built_years'],
      dtype='object')

In [32]:
# transform infinite data columns 
data_transform =data_transform[~data_transform.isin([np.nan, np.inf, -np.inf]).any(1)]

### Split dataset

In [33]:
Data_all = data_transform.copy()

X_data = Data_all.drop(['market_value'], axis=1)
y_data = Data_all.market_value

# divide data into training, validation and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=123)
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123)

### Preprocess using pipeline 

In [34]:
# Create transformer for numeric features
# Fill missing values with median and then scale
numeric_features = ['census_tract', 'depth', 'exempt_building', 'exempt_land','taxable_building', 'taxable_land',
                    'fireplaces', 'frontage', 'garage_spaces','geographic_ward','sale_price', 
                    'number_of_bathrooms','number_of_bedrooms', 'number_of_rooms', 'number_stories',
                    'total_area', 'total_livable_area','Sale_Record_dayDiff','Built_years','lat','lng']
numeric_transformer = Pipeline(steps=
                               [("fill_missing", SimpleImputer(strategy="median")), 
                                ("scaler", StandardScaler())
                               ])

# Create transformer for categorical features
# Fill missing values with mode and then one-hot encode
categorical_features = ['basements', 'category_code_description', 'central_air','exterior_condition',
                   'garage_type', 'general_construction', 'house_number','interior_condition',
                   'off_street_open', 'parcel_number', 'parcel_shape','site_type','view_type',
                   'street_designation','street_direction','topography','type_heater','zip_code','year_built_estimate']
# Convert all to strings
for feat in categorical_features:
    X_train_sub[feat]=X_train_sub[feat].astype(str)
# Create transformer pipeline for categorical data
categorical_transformer = Pipeline(steps=
                                   [("fill_missing",SimpleImputer(strategy='constant', fill_value='missing')),
                                    ("encoder",OneHotEncoder(handle_unknown="ignore"))
                                   ])

# Combine the two transformers into single ColumnTransformer preprocessor
preprocessor = ColumnTransformer(transformers=
                                 [("num", numeric_transformer, numeric_features),
                                  ("cat", categorical_transformer, categorical_features)])


In [47]:
# Create pipeline with preprocessor and model
model_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", LinearRegression())])

# Fit the pipeline on the training data
model_pipeline.fit(X_train_sub, y_train_sub)

# Use the pipeline to get predictions on test set and evaluate
val_preds = model_pipeline.predict(X_val)
r2 = r2_score(y_val,val_preds)
print("R-squared on test set: {:.3f}".format(r2))

accuracy = model_pipeline.score(X_val,y_val)
print("Accuracy on validation set: {:.3f}".format(accuracy))

MSE = mean_squared_error(y_val,val_preds)
print("MSE on validation set: {:.3f}".format(accuracy))

R-squared on test set: 1.000
Accuracy on validation set: 1.000
MSE on validation set: 1.000


### Model selection 

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor


clfs = []
clfs.append(LinearRegression())
clfs.append(Lasso())
clfs.append(Ridge())
clfs.append(ElasticNet())
clfs.append(DecisionTreeRegressor())
clfs.append(RandomForestRegressor())
clfs.append(KNeighborsRegressor())
clfs.append(GradientBoostingRegressor())


for classifier in clfs:
    model_pipeline.set_params(model = classifier)
    # cross-val_score on X_train, y_train
    model_pipeline.fit(X_train_sub, y_train_sub)
    accuracy = model_pipeline.score(X_val, y_val)

    print('---------------------------------')
    print(str(classifier))
    print('Accuracy for this classifier is', accuracy)

---------------------------------
LinearRegression()
Accuracy for this classifier is 0.9999999678646587
---------------------------------
Lasso()
Accuracy for this classifier is 0.9999999998655403
---------------------------------
Ridge()
Accuracy for this classifier is 0.9999981106891036
---------------------------------
ElasticNet()
Accuracy for this classifier is 0.9466641205467486
---------------------------------
DecisionTreeRegressor()
Accuracy for this classifier is 0.9920539679782204
---------------------------------
RandomForestRegressor()
Accuracy for this classifier is 0.9951284748709962
---------------------------------
KNeighborsRegressor()
Accuracy for this classifier is 0.9442350520157171
---------------------------------
GradientBoostingRegressor()
Accuracy for this classifier is 0.9955149799221633
